In [4]:
import os
import glob
import pandas as pd
import numpy as np
import math
import re
import csv
from datetime import datetime, timedelta
import re
from itertools import zip_longest
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


# Read and convert data to other time frame

In [5]:
# Read Daily data file and format columns
symbolDailyData = pd.read_csv('./Seasonality.csv')
symbolDailyData['Date'] = pd.to_datetime(symbolDailyData['Date'], format='%d-%m-%Y')
symbolDailyData['Weekday'] = symbolDailyData['Date'].dt.day_name()
symbolDailyData = symbolDailyData.set_index('Date').sort_index(ascending=True)


columnLogic = {
    'Ticker'       : 'first',
    'Open'         : 'first',
    'High'         : 'max',
    'Low'          : 'min',
    'Close'        : 'last',
    'Volume'       : 'sum',
    'OpenInterest' : 'last',
    'Weekday'      : 'first'
}


# Convert daily to weekly timeframe from monday as start of week
symbolMondayWeeklyData = symbolDailyData.resample('W-SUN').apply(columnLogic).reset_index()
symbolMondayWeeklyData['Date'] = symbolMondayWeeklyData['Date'] - pd.tseries.frequencies.to_offset("6D")
symbolMondayWeeklyData['Weekday'] = symbolMondayWeeklyData['Date'].dt.day_name()
symbolMondayWeeklyData = symbolMondayWeeklyData.set_index('Date').sort_index(ascending=True)


# Convert daily to weekly timeframe from expiry week and friday as start of week
symbolExpiryWeeklyData = symbolDailyData.resample('W-THU').apply(columnLogic).reset_index()
symbolExpiryWeeklyData['StartDate'] = symbolExpiryWeeklyData['Date'] - pd.tseries.frequencies.to_offset("6D")
symbolExpiryWeeklyData['Weekday'] = symbolExpiryWeeklyData['Date'].dt.day_name()
symbolExpiryWeeklyData = symbolExpiryWeeklyData.set_index('Date').sort_index(ascending=True)


# Convert daily to monthly timeframe
symbolMonthlyData = symbolDailyData.resample('M').apply(columnLogic).reset_index()
symbolMonthlyData['Date'] = pd.to_datetime(symbolMonthlyData['Date'].dt.strftime('%m-%Y'))
symbolMonthlyData['Weekday'] = symbolMonthlyData['Date'].dt.day_name()
symbolMonthlyData = symbolMonthlyData.set_index('Date').sort_index(ascending=True)


# Convert daily to yearly timeframe
symbolYearlyData = symbolDailyData.resample('Y').apply(columnLogic).reset_index()
symbolYearlyData['Date'] = pd.to_datetime(symbolYearlyData['Date'].dt.strftime('%Y'))
symbolYearlyData['Weekday'] = symbolYearlyData['Date'].dt.day_name()
symbolYearlyData = symbolYearlyData.set_index('Date').sort_index(ascending=True)


# Reset all dataframe indices
symbolDailyData.reset_index(inplace=True)
symbolMondayWeeklyData.reset_index(inplace=True)
symbolExpiryWeeklyData.reset_index(inplace=True)
symbolMonthlyData.reset_index(inplace=True)
symbolYearlyData.reset_index(inplace=True)




# functions defination

In [6]:


def getYearlyReturns(row):
    yearlyReturns = [np.nan, np.nan]
    yearlyRow = symbolYearlyData[ symbolYearlyData['Date'] == datetime(row['Date'].year, 1, 1) ]
    if ( len(yearlyRow) > 0 ):
        yearlyReturnPoints = yearlyRow['ReturnPoints'].iloc[0]
        yearlyReturnPercentage = yearlyRow['ReturnPercentage'].iloc[0]
        yearlyReturns[0] = yearlyReturnPoints
        yearlyReturns[1] = yearlyReturnPercentage
    return yearlyReturns


def getMonthlyReturns(row):
    monthlyReturns = [np.nan, np.nan]
    monthlyRow = symbolMonthlyData[ symbolMonthlyData['Date'] == datetime(row['Date'].year, row['Date'].month, 1) ]
    if ( len(monthlyRow) > 0 ):
        monthlyReturnPoints = monthlyRow['ReturnPoints'].iloc[0]
        monthlyReturnPercentage = monthlyRow['ReturnPercentage'].iloc[0]
        monthlyReturns[0] = monthlyReturnPoints
        monthlyReturns[1] = monthlyReturnPercentage
    return monthlyReturns


def getMondayWeeklyData(row):
    mondayWeeklyData = [np.nan, np.nan, np.nan, np.nan]
    mondayRow =  symbolMondayWeeklyData[ symbolMondayWeeklyData['Date'] == row['MondayWeeklyDate'] ]
    if ( len(mondayRow) > 0 ):
        mondayWeeklyReturnPoints = mondayRow['ReturnPoints'].iloc[0]
        mondayWeeklyReturnPercentage = mondayRow['ReturnPercentage'].iloc[0]
        mondayWeekNumberMonthly = mondayRow['WeekNumberMonthly'].iloc[0]
        mondayWeekNumberYearly = mondayRow['WeekNumberYearly'].iloc[0]
        mondayWeeklyData[0] = mondayWeeklyReturnPoints
        mondayWeeklyData[1] = mondayWeeklyReturnPercentage
        mondayWeeklyData[2] = mondayWeekNumberMonthly
        mondayWeeklyData[3] = mondayWeekNumberYearly
    return mondayWeeklyData


def getExpiryWeeklyData(row):
    expiryWeeklyData = [np.nan, np.nan, np.nan, np.nan]
    expiryRow =  symbolExpiryWeeklyData[ symbolExpiryWeeklyData['Date'] == row['ExpiryWeeklyDate'] ]
    if ( len(expiryRow) > 0 ):
        expiryWeeklyReturnPoints = expiryRow['ReturnPoints'].iloc[0]
        expiryWeeklyReturnPercentage = expiryRow['ReturnPercentage'].iloc[0]
        expiryWeekNumberMonthly = expiryRow['WeekNumberMonthly'].iloc[0]
        expiryWeekNumberYearly = expiryRow['WeekNumberYearly'].iloc[0]
        expiryWeeklyData[0] = expiryWeeklyReturnPoints
        expiryWeeklyData[1] = expiryWeeklyReturnPercentage
        expiryWeeklyData[2] = expiryWeekNumberMonthly
        expiryWeeklyData[3] = expiryWeekNumberYearly
    return expiryWeeklyData



# yearly data, <br>find returns odd/even values

In [7]:
symbolYearlyData['EvenYear'] = ((symbolYearlyData['Date'].dt.year % 2) == 0)
symbolYearlyData['ReturnPoints'] = symbolYearlyData['Close'] - symbolYearlyData['Close'].shift(1)
symbolYearlyData['ReturnPercentage'] = round((symbolYearlyData['ReturnPoints']/symbolYearlyData['Close'].shift(1)*100), 2)
symbolYearlyData['PositiveYear'] = (symbolYearlyData['ReturnPoints'] > 0)




# monthly data, <br>find returns and odd/even values

In [8]:
symbolMonthlyData['EvenMonth'] = ((symbolMonthlyData['Date'].dt.month % 2) == 0)
symbolMonthlyData['ReturnPoints'] = symbolMonthlyData['Close'] - symbolMonthlyData['Close'].shift(1)
symbolMonthlyData['ReturnPercentage'] = round(
    (symbolMonthlyData['ReturnPoints']/symbolMonthlyData['Close'].shift(1)*100), 2
)
symbolMonthlyData['PositiveMonth'] = (symbolMonthlyData['ReturnPoints'] > 0)

yearlyReturns = np.array(symbolMonthlyData.apply(lambda row: getYearlyReturns(row), axis=1).tolist()).transpose()
symbolMonthlyData['EvenYear'] = ((symbolMonthlyData['Date'].dt.year % 2) == 0)
symbolMonthlyData['YearlyReturnPoints'], symbolMonthlyData['YearlyReturnPercentage'] = yearlyReturns[0], yearlyReturns[1]
symbolMonthlyData['PositiveYear'] = (symbolMonthlyData['YearlyReturnPoints'] > 0)




# weekly data monday based, <br>find returns and odd/even values

In [9]:
symbolMondayWeeklyData['WeekNumberMonthly'] = symbolMondayWeeklyData['WeekNumberYearly'] = np.nan


for i in range(1, len(symbolMondayWeeklyData)):
    if ( symbolMondayWeeklyData.loc[i, 'Date'].month != symbolMondayWeeklyData.loc[i-1, 'Date'].month ):
        symbolMondayWeeklyData.loc[i, 'WeekNumberMonthly'] = 1
    else:
        symbolMondayWeeklyData.loc[i, 'WeekNumberMonthly'] = symbolMondayWeeklyData.loc[i-1, 'WeekNumberMonthly'] + 1
    if ( symbolMondayWeeklyData.loc[i, 'Date'].year != symbolMondayWeeklyData.loc[i-1, 'Date'].year ):
        symbolMondayWeeklyData.loc[i, 'WeekNumberYearly'] = 1
    else:
        symbolMondayWeeklyData.loc[i, 'WeekNumberYearly'] = symbolMondayWeeklyData.loc[i-1, 'WeekNumberYearly'] + 1


symbolMondayWeeklyData['EvenWeekNumberMonthly'] = ((symbolMondayWeeklyData['WeekNumberMonthly'] % 2) == 0)
symbolMondayWeeklyData['EvenWeekNumberYearly'] = ((symbolMondayWeeklyData['WeekNumberYearly'] % 2) == 0)
symbolMondayWeeklyData[['WeekNumberMonthly', 'WeekNumberYearly']] = symbolMondayWeeklyData[
    ['WeekNumberMonthly', 'WeekNumberYearly']
].fillna(value=0).astype('Int64').replace(0, np.nan)

symbolMondayWeeklyData['ReturnPoints'] = symbolMondayWeeklyData['Close'] - symbolMondayWeeklyData['Close'].shift(1)
symbolMondayWeeklyData['ReturnPercentage'] = round(
    (symbolMondayWeeklyData['ReturnPoints']/symbolMondayWeeklyData['Close'].shift(1)*100), 2
)
symbolMondayWeeklyData['PositiveWeek'] = (symbolMondayWeeklyData['ReturnPoints'] > 0)

monthlyReturns = np.array(symbolMondayWeeklyData.apply(lambda row: getMonthlyReturns(row), axis=1).tolist()).transpose()
symbolMondayWeeklyData['EvenMonth'] = ((symbolMondayWeeklyData['Date'].dt.month % 2) == 0)
symbolMondayWeeklyData['MonthlyReturnPoints'], symbolMondayWeeklyData['MonthlyReturnPercentage'] = monthlyReturns[0], monthlyReturns[1]
symbolMondayWeeklyData['PositiveMonth'] = (symbolMondayWeeklyData['MonthlyReturnPoints'] > 0)

yearlyReturns = np.array(symbolMondayWeeklyData.apply(lambda row: getYearlyReturns(row), axis=1).tolist()).transpose()
symbolMondayWeeklyData['EvenYear'] = ((symbolMondayWeeklyData['Date'].dt.year % 2) == 0)
symbolMondayWeeklyData['YearlyReturnPoints'], symbolMondayWeeklyData['YearlyReturnPercentage'] = yearlyReturns[0], yearlyReturns[1]
symbolMondayWeeklyData['PositiveYear'] = (symbolMondayWeeklyData['YearlyReturnPoints'] > 0)




# weekly data expiry based, <br>find returns and odd/even values

In [10]:
symbolExpiryWeeklyData['WeekNumberMonthly'] = symbolExpiryWeeklyData['WeekNumberYearly'] = np.nan


for i in range(1, len(symbolExpiryWeeklyData)):
    if ( symbolExpiryWeeklyData.loc[i, 'Date'].month != symbolExpiryWeeklyData.loc[i-1, 'Date'].month ):
        symbolExpiryWeeklyData.loc[i, 'WeekNumberMonthly'] = 1
    else:
        symbolExpiryWeeklyData.loc[i, 'WeekNumberMonthly'] = symbolExpiryWeeklyData.loc[i-1, 'WeekNumberMonthly'] + 1
    if ( symbolExpiryWeeklyData.loc[i, 'Date'].year != symbolExpiryWeeklyData.loc[i-1, 'Date'].year ):
        symbolExpiryWeeklyData.loc[i, 'WeekNumberYearly'] = 1
    else:
        symbolExpiryWeeklyData.loc[i, 'WeekNumberYearly'] = symbolExpiryWeeklyData.loc[i-1, 'WeekNumberYearly'] + 1


symbolExpiryWeeklyData['EvenWeekNumberMonthly'] = ((symbolExpiryWeeklyData['WeekNumberMonthly'] % 2) == 0)
symbolExpiryWeeklyData['EvenWeekNumberYearly'] = ((symbolExpiryWeeklyData['WeekNumberYearly'] % 2) == 0)
symbolExpiryWeeklyData[['WeekNumberMonthly', 'WeekNumberYearly']] = symbolExpiryWeeklyData[
    ['WeekNumberMonthly', 'WeekNumberYearly']
].fillna(value=0).astype('Int64').replace(0, np.nan)

symbolExpiryWeeklyData['ReturnPoints'] = symbolExpiryWeeklyData['Close'] - symbolExpiryWeeklyData['Close'].shift(1)
symbolExpiryWeeklyData['ReturnPercentage'] = round(
    (symbolExpiryWeeklyData['ReturnPoints']/symbolExpiryWeeklyData['Close'].shift(1)*100), 2
)
symbolExpiryWeeklyData['PositiveWeek'] = (symbolExpiryWeeklyData['ReturnPoints'] > 0)

monthlyReturns = np.array(symbolExpiryWeeklyData.apply(lambda row: getMonthlyReturns(row), axis=1).tolist()).transpose()
symbolExpiryWeeklyData['EvenMonth'] = ((symbolExpiryWeeklyData['Date'].dt.month % 2) == 0)
symbolExpiryWeeklyData['MonthlyReturnPoints'], symbolExpiryWeeklyData['MonthlyReturnPercentage'] = monthlyReturns[0], monthlyReturns[1]
symbolExpiryWeeklyData['PositiveMonth'] = (symbolExpiryWeeklyData['MonthlyReturnPoints'] > 0)

yearlyReturns = np.array(symbolExpiryWeeklyData.apply(lambda row: getYearlyReturns(row), axis=1).tolist()).transpose()
symbolExpiryWeeklyData['EvenYear'] = ((symbolExpiryWeeklyData['Date'].dt.year % 2) == 0)
symbolExpiryWeeklyData['YearlyReturnPoints'], symbolExpiryWeeklyData['YearlyReturnPercentage'] = yearlyReturns[0], yearlyReturns[1]
symbolExpiryWeeklyData['PositiveYear'] = (symbolExpiryWeeklyData['YearlyReturnPoints'] > 0)




# daily data, <br>find returns, odd/even values and calender/trading days

In [11]:
symbolDailyData['CalenderMonthDay'] = symbolDailyData['Date'].dt.day
symbolDailyData['CalenderYearDay'] = symbolDailyData['Date'].dt.dayofyear
symbolDailyData['TradingMonthDay'] = symbolDailyData['TradingYearDay'] = np.nan


for i in range(1, len(symbolDailyData)):
    if ( symbolDailyData.loc[i, 'Date'].month != symbolDailyData.loc[i-1, 'Date'].month ):
        symbolDailyData.loc[i, 'TradingMonthDay'] = 1
    else:
        symbolDailyData.loc[i, 'TradingMonthDay'] = symbolDailyData.loc[i-1, 'TradingMonthDay'] + 1
    if ( symbolDailyData.loc[i, 'Date'].year != symbolDailyData.loc[i-1, 'Date'].year ):
        symbolDailyData.loc[i, 'TradingYearDay'] = 1
    else:
        symbolDailyData.loc[i, 'TradingYearDay'] = symbolDailyData.loc[i-1, 'TradingYearDay'] + 1


symbolDailyData['EvenCalenderMonthDay'] = ((symbolDailyData['CalenderMonthDay'] % 2) == 0)
symbolDailyData['EvenCalenderYearDay'] = ((symbolDailyData['CalenderYearDay'] % 2) == 0)
symbolDailyData['EvenTradingMonthDay'] = ((symbolDailyData['TradingMonthDay'] % 2) == 0)
symbolDailyData['EvenTradingYearDay'] = ((symbolDailyData['TradingYearDay'] % 2) == 0)

symbolDailyData[['TradingMonthDay', 'TradingYearDay']] = symbolDailyData[
    ['TradingMonthDay', 'TradingYearDay']
].fillna(value=0).astype('Int64').replace(0, np.nan)

symbolDailyData['ReturnPoints'] = symbolDailyData['Close'] - symbolDailyData['Close'].shift(1)
symbolDailyData['ReturnPercentage'] = round(
    (symbolDailyData['ReturnPoints']/symbolDailyData['Close'].shift(1)*100), 2
)
symbolDailyData['PositiveDay'] = (symbolDailyData['ReturnPoints'] > 0)


# weekly monday calculations
symbolDailyData['MondayWeeklyDate'] = symbolDailyData['Date'].apply(
    lambda x: x - pd.tseries.frequencies.to_offset(str(x.weekday()) + 'D')
)
mondayWeeklyData = np.array(
    symbolDailyData.apply(lambda row: getMondayWeeklyData(row), axis=1).tolist()
).transpose()

symbolDailyData['MondayWeekNumberMonthly'] = symbolDailyData['MondayWeekNumberYearly'] = np.nan
symbolDailyData['MondayWeekNumberMonthly'], symbolDailyData['MondayWeekNumberYearly'], \
symbolDailyData['EvenMondayWeekNumberMonthly'], symbolDailyData['EvenMondayWeekNumberYearly'], \
symbolDailyData['MondayWeeklyReturnPoints'], symbolDailyData['MondayWeeklyReturnPercentage'], \
symbolDailyData['PositiveMondayWeek'] = \
mondayWeeklyData[2], mondayWeeklyData[3], \
([((i % 2) == 0) for i in mondayWeeklyData[2]]), ([((i % 2) == 0) for i in mondayWeeklyData[3]]), \
mondayWeeklyData[0], mondayWeeklyData[1], \
([(i > 0) for i in mondayWeeklyData[0]])


# weekly expiry calculations
symbolDailyData['ExpiryWeeklyDate'] = symbolDailyData['Date'].apply(
    lambda x: (x + pd.tseries.frequencies.to_offset(str(6) + 'D')) if (x.weekday() == 4) 
    else (x + pd.tseries.frequencies.to_offset(str(3-x.weekday()) + 'D'))
)
expiryWeeklyData = np.array(
    symbolDailyData.apply(lambda row: getExpiryWeeklyData(row), axis=1).tolist()
).transpose()

symbolDailyData['ExpiryWeekNumberMonthly'] = symbolDailyData['ExpiryWeekNumberYearly'] = np.nan
symbolDailyData['ExpiryWeekNumberMonthly'], symbolDailyData['ExpiryWeekNumberYearly'], \
symbolDailyData['EvenExpiryWeekNumberMonthly'], symbolDailyData['EvenExpiryWeekNumberYearly'], \
symbolDailyData['ExpiryWeeklyReturnPoints'], symbolDailyData['ExpiryWeeklyReturnPercentage'], \
symbolDailyData['PositiveExpiryWeek'] = \
expiryWeeklyData[2], expiryWeeklyData[3], \
([((i % 2) == 0) for i in expiryWeeklyData[2]]), ([((i % 2) == 0) for i in expiryWeeklyData[3]]), \
expiryWeeklyData[0], expiryWeeklyData[1], \
([(i > 0) for i in expiryWeeklyData[0]])


# replace not a number values in data frame
symbolDailyData[['MondayWeekNumberMonthly', 'MondayWeekNumberYearly']] = symbolDailyData[
    ['MondayWeekNumberMonthly', 'MondayWeekNumberYearly']
].fillna(value=0).astype('Int64').replace(0, np.nan)
symbolDailyData[['ExpiryWeekNumberMonthly', 'ExpiryWeekNumberYearly']] = symbolDailyData[
    ['ExpiryWeekNumberMonthly', 'ExpiryWeekNumberYearly']
].fillna(value=0).astype('Int64').replace(0, np.nan)


# monthly data calculation
monthlyReturns = np.array(symbolDailyData.apply(lambda row: getMonthlyReturns(row), axis=1).tolist()).transpose()
symbolDailyData['EvenMonth'] = ((symbolDailyData['Date'].dt.month % 2) == 0)
symbolDailyData['MonthlyReturnPoints'], symbolDailyData['MonthlyReturnPercentage'] = monthlyReturns[0], monthlyReturns[1]
symbolDailyData['PositiveMonth'] = (symbolDailyData['MonthlyReturnPoints'] > 0)


# yearly data calculation
yearlyReturns = np.array(symbolDailyData.apply(lambda row: getYearlyReturns(row), axis=1).tolist()).transpose()
symbolDailyData['EvenYear'] = ((symbolDailyData['Date'].dt.year % 2) == 0)
symbolDailyData['YearlyReturnPoints'], symbolDailyData['YearlyReturnPercentage'] = yearlyReturns[0], yearlyReturns[1]
symbolDailyData['PositiveYear'] = (symbolDailyData['YearlyReturnPoints'] > 0)





In [12]:
# symbolDailyData.set_index('Date').to_csv('./1_Daily.csv')
# symbolMondayWeeklyData.set_index('Date').to_csv('./2_MondayWeekly.csv')
# symbolExpiryWeeklyData.set_index('Date').to_csv('./3_ExpiryWeekly.csv')
# symbolMonthlyData.set_index('Date').to_csv('./4_Monthly.csv')
# symbolYearlyData.set_index('Date').to_csv('./5_Yearly.csv')

# symbolDailyData.columns
# symbolMondayWeeklyData.columns
# symbolExpiryWeeklyData.columns
# symbolMonthlyData.columns
# symbolYearlyData.columns




In [13]:
getNotAvailableDates = lambda x, y: list((set(x)- set(y))) + list((set(y)- set(x))) 

tradingDates = symbolDailyData['Date'].to_list()
stratDate = symbolDailyData['Date'][0]
endDate = symbolDailyData['Date'][len(symbolDailyData['Date'])-1]
dateRange = [stratDate+timedelta(days=x) for x in range((endDate+timedelta(days=1)-stratDate).days)]
notAvailableDates = getNotAvailableDates(dateRange, tradingDates)

tradingDates[0]
tradingDates[len(tradingDates)-1]
dateRange[0]
dateRange[len(dateRange)-1]

type(tradingDates), type(tradingDates[0])
type(dateRange), type(dateRange[0])
type(notAvailableDates), type(notAvailableDates[0])

len(tradingDates)
len(dateRange)
len(notAvailableDates)

years = list(set(symbolDailyData['Date'].dt.year.to_list()))
max(years)
min(years)




Timestamp('2005-06-09 00:00:00')

Timestamp('2023-03-27 00:00:00')

Timestamp('2005-06-09 00:00:00')

Timestamp('2023-03-27 00:00:00')

(list, pandas._libs.tslibs.timestamps.Timestamp)

...

4414

6501

2087

2023

2005

In [13]:
x = symbolDailyData.dropna().groupby('Weekday')['ReturnPercentage'].mean().reset_index()
x

x = symbolDailyData.dropna().groupby('Weekday')['ReturnPercentage'].mean().reindex(
    ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
).reset_index()

x

y = ((x['ReturnPercentage']/100)+1).to_list()

z = [0] * len(y)
z[0] = y[0]

for i in range(1, len(y)):
    z[i] = z[i-1]*y[i]

x['SuperImposedReturn'] = z
x['SuperImposedReturn'] = (x['SuperImposedReturn']-1)*100

x



,Weekday,ReturnPercentage
0,Friday,0.107423
1,Monday,-0.008142
2,Thursday,0.020314
3,Tuesday,0.087644
4,Wednesday,0.094248


,Weekday,ReturnPercentage
0,Monday,-0.008142
1,Tuesday,0.087644
2,Wednesday,0.094248
3,Thursday,0.020314
4,Friday,0.107423


,Weekday,ReturnPercentage,SuperImposedReturn
0,Monday,-0.008142,-0.008142
1,Tuesday,0.087644,0.079495
2,Wednesday,0.094248,0.173818
3,Thursday,0.020314,0.194167
4,Friday,0.107423,0.301798


In [82]:
symbolDailyData['ExpiryWeeklyDate'] = symbolDailyData['Date'].apply(
    lambda x: (x + pd.tseries.frequencies.to_offset(str(6) + 'D')) if (x.weekday() == 4) 
    else (x + pd.tseries.frequencies.to_offset(str(3-x.weekday()) + 'D'))
)

symbolDailyData.head(25)




,Date,Ticker,Open,High,Low,Close,Volume,OpenInterest,Weekday,ExpiryWeeklyDate
0,2005-06-09,NIFTY BANK,3659.80,3664.45,3609.75,3636.5,0,0,Thursday,2005-06-09
1,2005-06-10,NIFTY BANK,3640.05,3665.80,3585.80,3593.5,0,0,Friday,2005-06-16
2,2005-06-13,NIFTY BANK,3593.90,3648.40,3556.70,3642.9,0,0,Monday,2005-06-16
3,2005-06-14,NIFTY BANK,3649.05,3696.35,3637.70,3689.8,0,0,Tuesday,2005-06-16
4,2005-06-15,NIFTY BANK,3700.80,3703.75,3662.85,3683.8,0,0,Wednesday,2005-06-16
5,2005-06-16,NIFTY BANK,3684.40,3685.75,3607.20,3618.8,0,0,Thursday,2005-06-16
6,2005-06-17,NIFTY BANK,3625.10,3627.45,3581.80,3595.8,0,0,Friday,2005-06-23
7,2005-06-20,NIFTY BANK,3600.90,3627.35,3518.95,3531.7,0,0,Monday,2005-06-23
8,2005-06-21,NIFTY BANK,3529.05,3587.10,3529.05,3577.5,0,0,Tuesday,2005-06-23
9,2005-06-22,NIFTY BANK,3578.65,3614.95,3546.95,3560.9,0,0,Wednesday,2005-06-23


In [49]:

symbolDailyData = symbolDailyData.dropna()

allReturns = np.array(symbolDailyData['ReturnPercentage'].to_list())
positiveReturns = allReturns[allReturns > 0]
negativeReturns = allReturns[allReturns < 0]

df1 = pd.DataFrame()
df1 = pd.concat([df1, pd.Series({
    'All Count': np.size(allReturns),
    'Positive Count': np.size(positiveReturns),
    'Negative Count': np.size(negativeReturns),
    'Average Return of All': np.mean(allReturns),
    'Average Return of Positive': np.mean(positiveReturns),
    'Average Return of Negative': np.mean(negativeReturns),
    'Sum Return of All': np.sum(allReturns),
    'Sum Return of Positive': np.sum(positiveReturns),
    'Sum Return of Negative': np.sum(negativeReturns)
})], axis=1)

allReturns = np.array(symbolDailyData[symbolDailyData['Weekday'] == 'Monday']['ReturnPercentage'].to_list())
positiveReturns = allReturns[allReturns > 0]
negativeReturns = allReturns[allReturns < 0]
    
df1 = pd.concat([df1, pd.Series({
    'All Count': np.size(allReturns),
    'Positive Count': np.size(positiveReturns),
    'Negative Count': np.size(negativeReturns),
    'Average Return of All': np.mean(allReturns),
    'Average Return of Positive': np.mean(positiveReturns),
    'Average Return of Negative': np.mean(negativeReturns),
    'Sum Return of All': np.sum(allReturns),
    'Sum Return of Positive': np.sum(positiveReturns),
    'Sum Return of Negative': np.sum(negativeReturns)
})], axis=1)

allReturns = np.array(symbolDailyData[symbolDailyData['Weekday'] == 'Friday']['ReturnPercentage'].to_list())
positiveReturns = allReturns[allReturns > 0]
negativeReturns = allReturns[allReturns < 0]

df1 = pd.concat([df1, pd.Series({
    'All Count': np.size(allReturns),
    'Positive Count': np.size(positiveReturns),
    'Negative Count': np.size(negativeReturns),
    'Average Return of All': np.mean(allReturns),
    'Average Return of Positive': np.mean(positiveReturns),
    'Average Return of Negative': np.mean(negativeReturns),
    'Sum Return of All': np.sum(allReturns),
    'Sum Return of Positive': np.sum(positiveReturns),
    'Sum Return of Negative': np.sum(negativeReturns)
})], axis=1)


df1.columns = ['All Days', 'Monday', 'Friday']

df1 = df1.T
df1 = df1.astype({
    'All Count': np.int64,
    'Positive Count': np.int64,
    'Negative Count': np.int64,
    'Average Return of All': np.float64,
    'Average Return of Positive': np.float64,
    'Average Return of Negative': np.float64,
    'Sum Return of All': np.float64,
    'Sum Return of Positive': np.float64,
    'Sum Return of Negative': np.float64
})

df1[[
    'Average Return of All', 'Average Return of Positive', 'Average Return of Negative', 
    'Sum Return of All', 'Sum Return of Positive', 'Sum Return of Negative'
]] = df1[[
    'Average Return of All', 'Average Return of Positive', 'Average Return of Negative', 
    'Sum Return of All', 'Sum Return of Positive', 'Sum Return of Negative'
]].round(decimals=2)

df1 = df1.T
df1



,All Days,Monday,Friday
All Count,4273.00,854.00,841.00
Positive Count,2247.00,462.00,445.00
Negative Count,2021.00,391.00,395.00
Average Return of All,0.07,-0.00,0.11
Average Return of Positive,1.27,1.30,1.36
Average Return of Negative,-1.27,-1.54,-1.31
Sum Return of All,288.94,-1.12,90.45
Sum Return of Positive,2856.82,601.46,607.38
Sum Return of Negative,-2567.88,-602.58,-516.93


In [50]:

def getAccuracy(row, weekDay):
    newRow = str(row[weekDay]) + '(' + str(round(row[weekDay]/row['All Days']*100, 2)) + '%)'
    return newRow

df1

df1['Monday'] = df1.apply(lambda x: getAccuracy(x, 'Monday'), axis=1)
df1['Friday'] = df1.apply(lambda x: getAccuracy(x, 'Friday'), axis=1)
df1





,All Days,Monday,Friday
All Count,4273.00,854.00,841.00
Positive Count,2247.00,462.00,445.00
Negative Count,2021.00,391.00,395.00
Average Return of All,0.07,-0.00,0.11
Average Return of Positive,1.27,1.30,1.36
Average Return of Negative,-1.27,-1.54,-1.31
Sum Return of All,288.94,-1.12,90.45
Sum Return of Positive,2856.82,601.46,607.38
Sum Return of Negative,-2567.88,-602.58,-516.93


,All Days,Monday,Friday
All Count,4273.00,854.0(19.99%),841.0(19.68%)
Positive Count,2247.00,462.0(20.56%),445.0(19.8%)
Negative Count,2021.00,391.0(19.35%),395.0(19.54%)
Average Return of All,0.07,-0.0(-0.0%),0.11(157.14%)
Average Return of Positive,1.27,1.3(102.36%),1.36(107.09%)
Average Return of Negative,-1.27,-1.54(121.26%),-1.31(103.15%)
Sum Return of All,288.94,-1.12(-0.39%),90.45(31.3%)
Sum Return of Positive,2856.82,601.46(21.05%),607.38(21.26%)
Sum Return of Negative,-2567.88,-602.58(23.47%),-516.93(20.13%)


In [9]:

dirct = [x[0] for x in os.walk('./Symbols')]

x = next(os.walk('./Symbols'))[1]
x
type(x)



['BANKNIFTY', 'NIFTY']

list

In [17]:
symbolMondayWeeklyDataNew = symbolMondayWeeklyData.copy(deep=True).dropna()
symbolMondayWeeklyDataNew.head(5)
symbolMondayWeeklyDataNew.columns

symbolMondayWeeklyDataNew['Date'] = pd.to_datetime(symbolMondayWeeklyDataNew['Date'], format='%Y-%m-%d')

symbolMondayWeeklyDataNew = symbolMondayWeeklyDataNew[ symbolMondayWeeklyDataNew['PositiveYear'] == True ]
symbolMondayWeeklyDataNew = symbolMondayWeeklyDataNew[ symbolMondayWeeklyDataNew['EvenYear'] == True ]
symbolMondayWeeklyDataNew




,Date,Ticker,Open,High,Low,Close,Volume,OpenInterest,Weekday,WeekNumberMonthly,...,ReturnPercentage,PositiveWeek,EvenMonth,MonthlyReturnPoints,MonthlyReturnPercentage,PositiveMonth,EvenYear,YearlyReturnPoints,YearlyReturnPercentage,PositiveYear
9,1995-01-09,NSENIFTY1,1112.10,1112.10,1066.19,1074.43,0,0.0,Monday,2,...,-4.95,False,False,-111.05,-9.39,False,False,-273.75,-23.15,False
10,1995-01-16,NSENIFTY1,1087.80,1091.57,1075.59,1075.59,0,0.0,Monday,3,...,0.11,True,False,-111.05,-9.39,False,False,-273.75,-23.15,False
11,1995-01-23,NSENIFTY1,1041.78,1043.59,1019.20,1043.59,0,0.0,Monday,4,...,-2.98,False,False,-111.05,-9.39,False,False,-273.75,-23.15,False
12,1995-01-30,NSENIFTY1,1058.60,1076.29,1058.60,1076.29,0,0.0,Monday,5,...,3.13,True,False,-111.05,-9.39,False,False,-273.75,-23.15,False
13,1995-02-06,NSENIFTY1,1061.72,1061.72,1037.16,1037.16,0,0.0,Monday,1,...,-3.64,False,True,-56.51,-5.28,False,False,-273.75,-23.15,False


Index(['Date', 'Ticker', 'Open', 'High', 'Low', 'Close', 'Volume',
       'OpenInterest', 'Weekday', 'WeekNumberMonthly', 'WeekNumberYearly',
       'EvenWeekNumberMonthly', 'EvenWeekNumberYearly', 'ReturnPoints',
       'ReturnPercentage', 'PositiveWeek', 'EvenMonth', 'MonthlyReturnPoints',
       'MonthlyReturnPercentage', 'PositiveMonth', 'EvenYear',
       'YearlyReturnPoints', 'YearlyReturnPercentage', 'PositiveYear'],
      dtype='object')

,Date,Ticker,Open,High,Low,Close,Volume,OpenInterest,Weekday,WeekNumberMonthly,...,ReturnPercentage,PositiveWeek,EvenMonth,MonthlyReturnPoints,MonthlyReturnPercentage,PositiveMonth,EvenYear,YearlyReturnPoints,YearlyReturnPercentage,PositiveYear
374,2002-01-07,NSENIFTY,1093.05,1119.40,1073.45,1088.55,344201360,0.0,Monday,1,...,-0.70,False,False,16.35,1.54,True,True,34.45,3.25,True
375,2002-01-14,NSENIFTY,1089.45,1121.75,1078.95,1093.15,343550116,0.0,Monday,2,...,0.42,True,False,16.35,1.54,True,True,34.45,3.25,True
376,2002-01-21,NSENIFTY,1093.25,1099.80,1077.00,1080.10,281384832,0.0,Monday,3,...,-1.19,False,False,16.35,1.54,True,True,34.45,3.25,True
377,2002-01-28,NSENIFTY1,1083.25,1088.75,1058.80,1081.65,283832980,0.0,Monday,4,...,0.14,True,False,16.35,1.54,True,True,34.45,3.25,True
378,2002-02-04,NSENIFTY1,1083.05,1131.85,1069.40,1123.75,367142188,0.0,Monday,1,...,3.89,True,True,66.65,6.20,True,True,34.45,3.25,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1464,2022-11-28,NSENIFTY,18430.55,18887.60,18365.60,18696.10,0,0.0,Monday,4,...,0.99,True,False,746.15,4.14,True,True,751.25,4.33,True
1465,2022-12-05,NSENIFTY,18719.55,18728.60,18410.10,18496.60,0,0.0,Monday,1,...,-1.07,False,True,-653.05,-3.48,False,True,751.25,4.33,True
1466,2022-12-12,NSENIFTY,18402.15,18696.10,18255.15,18269.00,0,0.0,Monday,2,...,-1.23,False,True,-653.05,-3.48,False,True,751.25,4.33,True
1467,2022-12-19,NSENIFTY,18288.10,18473.35,17779.50,17806.80,0,0.0,Monday,3,...,-2.53,False,True,-653.05,-3.48,False,True,751.25,4.33,True


In [27]:
xDate = symbolMondayWeeklyData['Date']
xDate

yDate = symbolMondayWeeklyDataNew['Date']
yDate

zDate = pd.Series(list(set(xDate).symmetric_difference(set(yDate))))
zDate




0      1994-11-07
1      1994-11-14
2      1994-11-21
3      1994-11-28
4      1994-12-05
          ...    
1477   2023-02-27
1478   2023-03-06
1479   2023-03-13
1480   2023-03-20
1481   2023-03-27
Name: Date, Length: 1482, dtype: datetime64[ns]

374    2002-01-07
375    2002-01-14
376    2002-01-21
377    2002-01-28
378    2002-02-04
          ...    
1464   2022-11-28
1465   2022-12-05
1466   2022-12-12
1467   2022-12-19
1468   2022-12-26
Name: Date, Length: 522, dtype: datetime64[ns]

0     2019-09-30
1     2008-03-17
2     1996-07-22
3     2013-12-09
4     2008-03-10
         ...    
955   2015-12-21
956   2008-08-04
957   1994-11-07
958   2009-08-03
959   2021-09-06
Length: 960, dtype: datetime64[ns]

In [27]:
weekDays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
tradingDays = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
calenderDays = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]

heatmap1DataList = []
totalTradingDays = []
yAxisName = ''
heatmap1TypeValue = 'TradingMonthDaysVsWeekdays'

if (heatmap1TypeValue == 'TradingMonthDaysVsWeekdays'):
    heatmap1DataList = [[0]*len(tradingDays)]*len(weekDays)
    totalTradingDays = tradingDays
    yAxisName = 'TradingMonthDay'
elif (heatmap1TypeValue == 'CalenderMonthDaysVsWeekdays'):
    heatmap1DataList = [[0]*len(calenderDays)]*len(weekDays)
    totalTradingDays = calenderDays
    yAxisName = 'CalenderMonthDay'

print(heatmap1DataList)

for index1, singleWeekDay in enumerate(weekDays):
    for index2, singleTradingDays in enumerate(totalTradingDays):
        meanValue = symbolDailyData[
            (symbolDailyData['Weekday'] == singleWeekDay) & (symbolDailyData[yAxisName] == singleTradingDays)
        ]['ReturnPercentage'].mean()
        heatmap1DataList[index1][index2] = meanValue
        print(singleWeekDay, singleTradingDays, index1, index2, heatmap1DataList[index1][index2])

print(heatmap1DataList)



[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
Monday 1 0 0 0.25578947368421057
Monday 2 0 1 0.24117647058823527
Monday 3 0 2 -0.07750000000000001
Monday 4 0 3 -0.021333333333333315
Monday 5 0 4 -0.13388888888888892
Monday 6 0 5 -0.09314285714285714
Monday 7 0 6 -0.00588235294117649
Monday 8 0 7 -0.13
Monday 9 0 8 -0.13529411764705881
Monday 10 0 9 -0.4045454545454545
Monday 11 0 10 0.07612903225806449
Monday 12 0 11 -0.13272727272727272
Monday 13 0 12 -0.035454545454545426
Monday 14 0 13 -0.1422727272727273
Monday 15 0 14 -0.8530434782608696
Monday 16 0 15 -0.3
Monday 17 0 16 -0.06749999999999996
Monday 18 0 17 -0.10846153846153844
Monday 19 0 18 0.08428571428571428
Monday 20 0 19 

In [13]:
symbolDailyData = symbolDailyData.dropna().reset_index().drop('index', axis=1)
data_csv = pd.read_csv('./SpecialDays/SpecialDays.csv').apply(pd.to_datetime, format='%d-%m-%Y', axis=1)
data_csv.columns = map(str.upper, data_csv.columns)

allSpecialDays = data_csv.columns.to_list()
selectedSpecialDay = allSpecialDays[3]

probabilityDays = 5
zerosList = [0.0]*probabilityDays

oldestDate = symbolDailyData['Date'][0]
earliestDate = symbolDailyData['Date'][len(symbolDailyData)-1]

allReturnsColumns = [
    *['Date(T)'],
    *[str('T-'+str(i)) for i in range(probabilityDays, 0, -1)], 
    *['T'], 
    *[str('T+'+str(i)) for i in range(1, probabilityDays+1)]
]

allReturns = pd.DataFrame(columns=allReturnsColumns)
selectedSpecialDates = [i for i in data_csv[selectedSpecialDay] if ((i > oldestDate) & (i < earliestDate))]


for i, specialDate in enumerate(selectedSpecialDates):
    
    beforeReturns, afterReturns = [], []
    
    specialDayReturns = symbolDailyData[(symbolDailyData['Date'] == selectedSpecialDates[i])]['ReturnPercentage'].to_list()
    if ( len(specialDayReturns) == 0 ):
        specialDayReturns = [0.0]
    
    if (i == 0):
        beforeReturns = symbolDailyData[
            (symbolDailyData['Date'] < selectedSpecialDates[i])
        ]['ReturnPercentage'][-probabilityDays:].to_list()
        afterReturns = symbolDailyData[
            (symbolDailyData['Date'] > selectedSpecialDates[i]) & (symbolDailyData['Date'] < selectedSpecialDates[i+1])
        ]['ReturnPercentage'][0:probabilityDays].to_list()
    
    elif (i+1 == len(selectedSpecialDates)):
        beforeReturns = symbolDailyData[
            (symbolDailyData['Date'] < selectedSpecialDates[i]) & ((symbolDailyData['Date'] > selectedSpecialDates[i-1]))
        ]['ReturnPercentage'][-probabilityDays:].to_list()
        afterReturns = symbolDailyData[
            (symbolDailyData['Date'] > selectedSpecialDates[i])
        ]['ReturnPercentage'][0:probabilityDays].to_list()
    
    else:
        beforeReturns = symbolDailyData[
            (symbolDailyData['Date'] < selectedSpecialDates[i]) & ((symbolDailyData['Date'] > selectedSpecialDates[i-1]))
        ]['ReturnPercentage'][-probabilityDays:].to_list()
        afterReturns = symbolDailyData[
            (symbolDailyData['Date'] > selectedSpecialDates[i]) & (symbolDailyData['Date'] < selectedSpecialDates[i+1])
        ]['ReturnPercentage'][0:probabilityDays].to_list()
    
    returnsColumn = [
        *[specialDate], 
        *[x+y for x,y in zip_longest(reversed(beforeReturns), reversed(zerosList), fillvalue=0)][::-1], 
        *specialDayReturns, 
        *[x+y for x,y in zip_longest(afterReturns, zerosList, fillvalue=0)]
    ]
    
    allReturns = pd.concat([
        allReturns, 
        pd.DataFrame({value:[returnsColumn[i]] for i, value in enumerate(allReturnsColumns)})
    ])


averageReturns = [round(i, 2) for i in allReturns[allReturnsColumns[1:]].mean()]
superimposedReturns = [(i/100)+1 for i in averageReturns]


for i in range(0, probabilityDays+1):
    if ( i == 0 ):
        superimposedReturns[probabilityDays] = superimposedReturns[probabilityDays]
    else:
        superimposedReturns[probabilityDays-i] = (
            superimposedReturns[probabilityDays-i]*superimposedReturns[probabilityDays-i+1]
        )
        superimposedReturns[probabilityDays+i] = (
            superimposedReturns[probabilityDays+i]*superimposedReturns[probabilityDays+i-1]
        )


superimposedReturns = [round((i-1)*100, 2) for i in superimposedReturns]
allReturns['Date(T)'] = allReturns['Date(T)'].dt.strftime('%d-%m-%Y')

allReturns = pd.concat([
    allReturns, 
    pd.DataFrame({value:[[*['Average Returns'], *averageReturns][i]] for i, value in enumerate(allReturnsColumns)})
])
allReturns = pd.concat([
    allReturns, 
    pd.DataFrame({value:[[*['Superimposed Returns'], *superimposedReturns][i]] for i, value in enumerate(allReturnsColumns)})
])

# allReturns.set_index('Date(T)').to_csv('./' + selectedSpecialDay + '.csv')

selectedSpecialDay
allReturns.head(10)




'HOLI'

,Date(T),T-5,T-4,T-3,T-2,T-1,T,T+1,T+2,T+3,T+4,T+5
0,24-03-2016,-0.17,1.48,1.35,0.13,-0.11,0.0,-0.97,-0.49,1.90,0.14,-0.46
0,13-03-2017,0.68,-0.20,-0.24,0.04,0.10,0.0,1.53,0.10,0.82,-0.06,-0.25
0,02-03-2018,1.11,0.80,-0.24,-0.60,-0.36,0.0,-0.89,-1.29,-0.59,0.72,-0.06
0,21-03-2019,0.04,0.75,0.42,0.53,-0.20,0.0,-0.47,-0.98,1.25,-0.52,1.29
0,10-03-2020,1.61,-0.64,0.02,-2.46,-4.77,0.0,-0.03,-7.88,4.05,-8.19,-2.55
0,29-03-2021,0.09,0.41,-1.64,-1.50,1.20,0.0,2.31,-1.09,1.18,-1.46,0.35
0,18-03-2022,0.33,1.60,-1.43,1.93,1.96,0.0,-1.07,1.26,-0.61,-0.15,-0.11
0,Average Returns,0.53,0.60,-0.25,-0.28,-0.31,0.0,0.06,-1.48,1.14,-1.36,-0.26
0,Superimposed Returns,0.29,-0.24,-0.84,-0.59,-0.31,0.0,0.06,-1.42,-0.30,-1.65,-1.91


In [15]:

nThDayBeforeSpecialDay = 3
allReturnsColumns = [
    *['T'+str(i) for i in range(-(nThDayBeforeSpecialDay-1), 0)],
    *['T'],
    *['T+'+str(i) for i in range(1, probabilityDays+1)]
]
allReturnsColumns = [*['Date(T-'+str(nThDayBeforeSpecialDay)+')'], *allReturnsColumns]
nThDayBeforeSpecialDayClosePrices = pd.DataFrame(columns=allReturnsColumns)


for i, specialDate in enumerate(selectedSpecialDates):
    
    specialDayReturns = symbolDailyData[(symbolDailyData['Date'] == selectedSpecialDates[i])][['Date', 'Close']]
    specialDayReturnsTemp = pd.DataFrame(pd.Series({'Date': selectedSpecialDates[i], 'Close': 0.0})).T
    if ( len(specialDayReturns) == 0 ):
        specialDayReturns = specialDayReturnsTemp.copy(deep=True)
    
    if (i == 0):
        beforeReturns = symbolDailyData[
            (symbolDailyData['Date'] < selectedSpecialDates[i])
        ][['Date', 'Close']][-nThDayBeforeSpecialDay:]
        afterReturns = symbolDailyData[
            (symbolDailyData['Date'] > selectedSpecialDates[i]) & (symbolDailyData['Date'] < selectedSpecialDates[i+1])
        ][['Date', 'Close']][0:probabilityDays]
    
    elif (i+1 == len(selectedSpecialDates)):
        beforeReturns = symbolDailyData[
            (symbolDailyData['Date'] < selectedSpecialDates[i]) & ((symbolDailyData['Date'] > selectedSpecialDates[i-1]))
        ][['Date', 'Close']][-nThDayBeforeSpecialDay:]
        afterReturns = symbolDailyData[
            (symbolDailyData['Date'] > selectedSpecialDates[i])
        ][['Date', 'Close']][0:probabilityDays]
    
    else:
        beforeReturns = symbolDailyData[
            (symbolDailyData['Date'] < selectedSpecialDates[i]) & ((symbolDailyData['Date'] > selectedSpecialDates[i-1]))
        ][['Date', 'Close']][-nThDayBeforeSpecialDay:]
        afterReturns = symbolDailyData[
            (symbolDailyData['Date'] > selectedSpecialDates[i]) & (symbolDailyData['Date'] < selectedSpecialDates[i+1])
        ][['Date', 'Close']][0:probabilityDays]
    
    if ( len(beforeReturns) < nThDayBeforeSpecialDay ):
        continue
    
    tempDataFrame = pd.concat([beforeReturns, specialDayReturns, afterReturns])
    nThDayDate, nThDayClose = tempDataFrame.iloc[0]
    nextCumulativePercentageChange = [
        None if j==0.0 else round(((j-nThDayClose)/nThDayClose)*100, 2) for j_i, j in enumerate(tempDataFrame['Close'][1:])
    ]
    customRow = [*[nThDayDate], *nextCumulativePercentageChange]
    
    nThDayBeforeSpecialDayClosePrices = pd.concat([
        nThDayBeforeSpecialDayClosePrices, 
        pd.DataFrame({value:[customRow[k]] for k, value in enumerate(allReturnsColumns)})
    ])
    
    
if ( nThDayBeforeSpecialDay > 1 ):    
    nThDayBeforeSpecialDayClosePrices = nThDayBeforeSpecialDayClosePrices.fillna(method='ffill', axis=1)
else:
    nThDayBeforeSpecialDayClosePrices = nThDayBeforeSpecialDayClosePrices.fillna(0.0)

nThDayBeforeSpecialDayClosePrices




,Date(T-3),T-2,T-1,T,T+1,T+2,T+3,T+4,T+5
0,2016-03-21,0.13,0.02,0.02,-0.95,-1.44,0.43,0.58,0.12
0,2017-03-08,0.04,0.15,0.15,1.68,1.78,2.62,2.56,2.31
0,2018-02-27,-0.6,-0.96,-0.96,-1.84,-3.1,-3.67,-2.98,-3.04
0,2019-03-18,0.53,0.32,0.32,-0.15,-1.12,0.12,-0.4,0.89
0,2020-03-05,-2.46,-7.11,-7.11,-7.14,-14.46,-10.99,-18.28,-20.37
0,2021-03-24,-1.5,-0.32,-0.32,1.99,0.88,2.07,0.58,0.93
0,2022-03-15,1.93,3.93,3.93,2.82,4.11,3.48,3.32,3.2


In [11]:
sumReturnOfAll = [-1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, -12, -13]
trendTypeValue = 'Bullish'
trendTypeMultiplier = 1 if (trendTypeValue=='Bullish') else -1
consecutiveTrendingDaysValue = 3
sumReturnValueChunks = []
sumReturnIndexChunks = []

idx = 0
traverseTill = len(sumReturnOfAll)-consecutiveTrendingDaysValue

while ( idx <= traverseTill ):
    checkChunkValues = False
    if ( (sumReturnOfAll[idx]*trendTypeMultiplier) > 0 ):
        checkChunkValues = all(
            (num*trendTypeMultiplier) > 0 for num in sumReturnOfAll[idx:(idx+consecutiveTrendingDaysValue)]
        )
    if (checkChunkValues):
        idx, sumReturnOfAll[idx:(idx+consecutiveTrendingDaysValue)]
        sumReturnValueChunks = [*sumReturnValueChunks, sumReturnOfAll[idx:(idx+consecutiveTrendingDaysValue)]]
        sumReturnIndexChunks = [*sumReturnIndexChunks, [idx, idx+consecutiveTrendingDaysValue-1]]
        idx = idx + consecutiveTrendingDaysValue
    else:
        idx = idx + 1


sumReturnValueChunks
sumReturnIndexChunks


(1, [2, 3, 4])

(4, [5, 6, 7])

(7, [8, 9, 10])

[[2, 3, 4], [5, 6, 7], [8, 9, 10]]

[[1, 3], [4, 6], [7, 9]]

In [44]:
df = pd.DataFrame({
    'A':[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13], 
    'B': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, -12, -13]
})
allScannerSymbolsData = pd.DataFrame()

symbolDailyData.head()
symbolDailyData.dropna(inplace=True)
symbolDailyData.head()


for sumReturnIndexChunk in sumReturnIndexChunks:
    sumReturnIndexChunk[0], sumReturnIndexChunk[1],
    type(symbolDailyData.iloc[sumReturnIndexChunk[0]:sumReturnIndexChunk[1]+1, :])
    symbolDailyData.iloc[sumReturnIndexChunk[0]:sumReturnIndexChunk[1]+1, :]
    allScannerSymbolsData = pd.concat(
        [allScannerSymbolsData, df.iloc[sumReturnIndexChunk[0]:sumReturnIndexChunk[1]+1, :]], axis=0
    )

allScannerSymbolsData

,Date,Ticker,Open,High,Low,Close,Volume,OpenInterest,Weekday,CalenderMonthDay,...,ExpiryWeeklyReturnPercentage,PositiveExpiryWeek,EvenMonth,MonthlyReturnPoints,MonthlyReturnPercentage,PositiveMonth,EvenYear,YearlyReturnPoints,YearlyReturnPercentage,PositiveYear
240,2016-01-04,NIFTY 50 1min out,7906.55,7937.45,7781.50,7789.15,0,0,Monday,4,...,-4.66,False,False,-380.35,-4.79,False,True,234.5,2.95,True
241,2016-01-05,NIFTY 50 1min out,7827.60,7831.10,7763.30,7779.00,0,0,Tuesday,5,...,-4.66,False,False,-380.35,-4.79,False,True,234.5,2.95,True
242,2016-01-06,NIFTY 50 1min out,7765.95,7800.95,7721.20,7733.40,0,0,Wednesday,6,...,-4.66,False,False,-380.35,-4.79,False,True,234.5,2.95,True
243,2016-01-07,NIFTY 50 1min out,7641.95,7655.50,7557.25,7569.65,0,0,Thursday,7,...,-4.66,False,False,-380.35,-4.79,False,True,234.5,2.95,True
244,2016-01-08,NIFTY 50 1min out,7616.00,7633.25,7581.15,7607.15,0,0,Friday,8,...,-0.38,False,False,-380.35,-4.79,False,True,234.5,2.95,True


,Date,Ticker,Open,High,Low,Close,Volume,OpenInterest,Weekday,CalenderMonthDay,...,ExpiryWeeklyReturnPercentage,PositiveExpiryWeek,EvenMonth,MonthlyReturnPoints,MonthlyReturnPercentage,PositiveMonth,EvenYear,YearlyReturnPoints,YearlyReturnPercentage,PositiveYear
240,2016-01-04,NIFTY 50 1min out,7906.55,7937.45,7781.50,7789.15,0,0,Monday,4,...,-4.66,False,False,-380.35,-4.79,False,True,234.5,2.95,True
241,2016-01-05,NIFTY 50 1min out,7827.60,7831.10,7763.30,7779.00,0,0,Tuesday,5,...,-4.66,False,False,-380.35,-4.79,False,True,234.5,2.95,True
242,2016-01-06,NIFTY 50 1min out,7765.95,7800.95,7721.20,7733.40,0,0,Wednesday,6,...,-4.66,False,False,-380.35,-4.79,False,True,234.5,2.95,True
243,2016-01-07,NIFTY 50 1min out,7641.95,7655.50,7557.25,7569.65,0,0,Thursday,7,...,-4.66,False,False,-380.35,-4.79,False,True,234.5,2.95,True
244,2016-01-08,NIFTY 50 1min out,7616.00,7633.25,7581.15,7607.15,0,0,Friday,8,...,-0.38,False,False,-380.35,-4.79,False,True,234.5,2.95,True


(1, 3)

pandas.core.frame.DataFrame

,Date,Ticker,Open,High,Low,Close,Volume,OpenInterest,Weekday,CalenderMonthDay,...,ExpiryWeeklyReturnPercentage,PositiveExpiryWeek,EvenMonth,MonthlyReturnPoints,MonthlyReturnPercentage,PositiveMonth,EvenYear,YearlyReturnPoints,YearlyReturnPercentage,PositiveYear
241,2016-01-05,NIFTY 50 1min out,7827.60,7831.10,7763.30,7779.00,0,0,Tuesday,5,...,-4.66,False,False,-380.35,-4.79,False,True,234.5,2.95,True
242,2016-01-06,NIFTY 50 1min out,7765.95,7800.95,7721.20,7733.40,0,0,Wednesday,6,...,-4.66,False,False,-380.35,-4.79,False,True,234.5,2.95,True
243,2016-01-07,NIFTY 50 1min out,7641.95,7655.50,7557.25,7569.65,0,0,Thursday,7,...,-4.66,False,False,-380.35,-4.79,False,True,234.5,2.95,True


...

,Date,Ticker,Open,High,Low,Close,Volume,OpenInterest,Weekday,CalenderMonthDay,...,ExpiryWeeklyReturnPercentage,PositiveExpiryWeek,EvenMonth,MonthlyReturnPoints,MonthlyReturnPercentage,PositiveMonth,EvenYear,YearlyReturnPoints,YearlyReturnPercentage,PositiveYear
244,2016-01-08,NIFTY 50 1min out,7616.00,7633.25,7581.15,7607.15,0,0,Friday,8,...,-0.38,False,False,-380.35,-4.79,False,True,234.5,2.95,True
245,2016-01-11,NIFTY 50 1min out,7497.25,7604.75,7494.55,7572.10,0,0,Monday,11,...,-0.38,False,False,-380.35,-4.79,False,True,234.5,2.95,True
246,2016-01-12,NIFTY 50 1min out,7574.70,7578.50,7488.00,7510.65,0,0,Tuesday,12,...,-0.38,False,False,-380.35,-4.79,False,True,234.5,2.95,True


(7, 9)

pandas.core.frame.DataFrame

,Date,Ticker,Open,High,Low,Close,Volume,OpenInterest,Weekday,CalenderMonthDay,...,ExpiryWeeklyReturnPercentage,PositiveExpiryWeek,EvenMonth,MonthlyReturnPoints,MonthlyReturnPercentage,PositiveMonth,EvenYear,YearlyReturnPoints,YearlyReturnPercentage,PositiveYear
247,2016-01-13,NIFTY 50 1min out,7562.15,7590.85,7426.10,7556.65,0,0,Wednesday,13,...,-0.38,False,False,-380.35,-4.79,False,True,234.5,2.95,True
248,2016-01-14,NIFTY 50 1min out,7457.70,7604.45,7444.15,7541.00,0,0,Thursday,14,...,-0.38,False,False,-380.35,-4.79,False,True,234.5,2.95,True
249,2016-01-15,NIFTY 50 1min out,7536.40,7562.50,7427.45,7433.95,0,0,Friday,15,...,-3.46,False,False,-380.35,-4.79,False,True,234.5,2.95,True


,A,B
1,3,3
2,4,4
3,5,5
4,6,6
5,7,7
6,8,8
7,9,9
8,10,10
9,11,11


In [46]:
symbolDailyData = symbolDailyData.dropna()
phenomenaDataTable = pd.DataFrame()

phenomenaMonthValue = 8
phenomenaDaysValue = [2, 9]

phenomenaDaysValueStart, phenomenaDaysValueEnd = phenomenaDaysValue[0], phenomenaDaysValue[1]
phenomenaGetMonthMultipier = 1 if phenomenaDaysValueStart<0 else -1
phenomenaGetMonthFilter = 1 if((phenomenaDaysValueStart < 0) & (phenomenaMonthValue == 12)) else 12 if((phenomenaDaysValueStart > 0) & (phenomenaMonthValue == 1)) else phenomenaMonthValue+(1*phenomenaGetMonthMultipier)
requiredDataForPhenomena = pd.DataFrame()

requiredDataForPhenomena = symbolDailyData[
    (symbolDailyData['Date'].dt.month == phenomenaMonthValue) |
    (symbolDailyData['Date'].dt.month == phenomenaGetMonthFilter)
]
lastTradingMonthDays = requiredDataForPhenomena[
    requiredDataForPhenomena['Date'].dt.month != requiredDataForPhenomena['Date'].shift(-1).dt.month
]['Date'].to_list()


for lastTradingMonthDay in lastTradingMonthDays:
    tempDataframe = pd.DataFrame()
    tempYear = lastTradingMonthDay.year
    tempStartClosing, tempEndClosing = 0, 0
    if ( phenomenaDaysValueStart < 0 ):
        if ( lastTradingMonthDay.month != phenomenaMonthValue ):
            continue
        tempDataframe = pd.concat([ 
            requiredDataForPhenomena[requiredDataForPhenomena['Date'] <= lastTradingMonthDay][phenomenaDaysValueStart:],
            requiredDataForPhenomena[requiredDataForPhenomena['Date'] > lastTradingMonthDay][0:phenomenaDaysValueEnd]
        ])
    else:
        if ( lastTradingMonthDay.month == phenomenaMonthValue ):
            continue
        tempDataframe = requiredDataForPhenomena[requiredDataForPhenomena['Date'] > lastTradingMonthDay][phenomenaDaysValueStart-1:phenomenaDaysValueEnd]
    tempStartClosing, tempEndClosing = tempDataframe['Close'].iloc[0], tempDataframe['Close'].iloc[-1]
    percentageChange = round(((tempEndClosing/tempStartClosing)-1)*100, 2)
    
    phenomenaDataTable = pd.concat([
        phenomenaDataTable,
        pd.DataFrame({'Years': [tempYear], 'Returns': [percentageChange]})
    ])


phenomenaDataTable

    


,Years,Returns
0,2016,-0.39
0,2017,-3.43
0,2018,1.11
0,2019,0.12
0,2020,1.89
0,2021,1.48
0,2022,2.33


In [11]:
def historicTrendingDays(listValues, _historicTrendType, _historicTrendConsecutiveDays, _historicTrendDayRange):
    index, consecutiveValuesIndex = _historicTrendDayRange, 0
    trendMultiplier = 1 if _historicTrendType=='Bullish' else -1
    listValuesIndex = [0]*len(listValues)
    consecutiveValuesIndexList = [i for i in range(1, _historicTrendConsecutiveDays+1)]
    
    while (index <= (len(listValues)-_historicTrendDayRange-1)):
        if ( (trendMultiplier*listValues[index]) > 0 ):
            consecutiveValuesIndex = consecutiveValuesIndex + 1
            if ( consecutiveValuesIndex == _historicTrendConsecutiveDays ):
                consecutiveValuesIndex = 0
                listValuesIndex[index+1-_historicTrendConsecutiveDays:index+1] = consecutiveValuesIndexList
                index = index + _historicTrendDayRange + 1
            else:
                index = index + 1
        else:
            consecutiveValuesIndex = 0
            index = index + 1
    return listValuesIndex


historicTrendDayData = symbolDailyData.dropna().copy(deep=True)
historicTrendType = 'Bearish'
historicTrendConsecutiveDays = 7
historicTrendDayRange = 7
historicTrendColumns = ['Date(T)', *['T'+str(i) for i in range(-historicTrendDayRange, 0)], *['T+'+str(i) for i in range(1, historicTrendDayRange+1)]]
totalReturns = []

historicTrendDayData['ConsecutiveDays'] = historicTrendingDays(
    historicTrendDayData['ReturnPoints'].to_list(), 
    historicTrendType, historicTrendConsecutiveDays, historicTrendDayRange
)
historicTrendDayData = historicTrendDayData[
    (historicTrendDayData['ConsecutiveDays'] == 0) | (historicTrendDayData['ConsecutiveDays'] == 1)
]
firstStreakDates = historicTrendDayData[historicTrendDayData['ConsecutiveDays'] == 1]['Date'].to_list()


for firstStreakDate in firstStreakDates:
    
    beforeDaysReturn = historicTrendDayData[historicTrendDayData['Date'] <= firstStreakDate][-historicTrendDayRange:]['ReturnPercentage'].to_list()
    afterDaysReturn = historicTrendDayData[historicTrendDayData['Date'] > firstStreakDate][:historicTrendDayRange]['ReturnPercentage'].to_list()
    totalReturns = [*totalReturns, [firstStreakDate, *beforeDaysReturn, *afterDaysReturn]]


historicTrendDataTable = pd.DataFrame(totalReturns, columns=historicTrendColumns)

historicTrendDataTable




,Date(T),T-7,T-6,T-5,T-4,T-3,T-2,T-1,T+1,T+2,T+3,T+4,T+5,T+6,T+7
0,2016-12-14,-0.01,-0.30,1.59,0.15,-1.01,0.50,-0.37,0.03,-0.97,1.50,0.07,0.85,0.85,0.06
1,2017-09-19,0.82,0.82,-0.02,0.01,-0.02,0.68,-0.05,0.22,0.23,0.73,0.71,-0.38,1.01,0.01
2,2018-01-30,0.72,0.54,1.14,0.01,-0.18,0.54,-0.68,1.15,-1.06,0.65,-0.71,0.86,-0.93,-0.61
3,2019-02-07,-0.36,1.66,0.70,0.15,0.24,1.21,-0.13,-0.91,-0.25,1.29,0.46,-0.02,0.83,-0.46
4,2019-04-30,-0.41,-1.32,-0.11,1.26,-0.62,0.83,-0.08,-0.36,-0.91,0.76,-0.86,1.19,1.26,3.64
5,2020-02-20,0.77,-0.29,-0.46,-0.52,-0.35,1.02,-0.39,1.61,-0.64,0.02,-2.46,-4.77,-0.03,-7.88
6,2022-02-16,0.18,1.34,0.66,-1.37,-3.03,3.16,-0.34,2.78,0.72,-0.97,-0.70,-1.61,-2.23,0.84
7,2022-04-29,1.46,-1.31,-1.10,1.29,-0.95,1.14,-0.69,-0.03,-2.32,-0.37,0.62,2.64,-0.16,-2.63


In [25]:
monthFullNames = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'Octomber', 'November', 'December']
symbolMonthlyData['Date'] = pd.to_datetime(symbolMonthlyData['Date'], format='%Y-%m-%d')
historicTrendDayData = symbolMonthlyData.dropna().copy(deep=True)

historicallyTrendingMonthName = 1
historicallyTrendingMonthTrend = 'Bullis'
historicallyTrendingMonthTrendMultiplier = True if historicallyTrendingMonthTrend=='Bullish' else False

historicallyTrendingMonthsData = historicTrendDayData[
    (historicTrendDayData['Date'].dt.month == historicallyTrendingMonthName) &
    (historicTrendDayData['PositiveMonth'] == historicallyTrendingMonthTrendMultiplier)
][['Date', 'Close', 'ReturnPercentage']]

nextMonthNumber = 12 if ((historicallyTrendingMonthName+1)%12)== 0 else (historicallyTrendingMonthName+1)%12
nextThreeMonthNumber = 12 if ((historicallyTrendingMonthName+3)%12)== 0 else (historicallyTrendingMonthName+3)%12
nextSixMonthNumber = 12 if ((historicallyTrendingMonthName+6)%12)== 0 else (historicallyTrendingMonthName+6)%12
nextTwelveMonthNumber = 12 if ((historicallyTrendingMonthName+12)%12)== 0 else (historicallyTrendingMonthName+12)%12

historicallyTrendingMonthsDataList = []
columnNames = [
    'Year', monthFullNames[historicallyTrendingMonthName-1], monthFullNames[nextMonthNumber-1],
    monthFullNames[nextMonthNumber-1]+'-'+monthFullNames[nextThreeMonthNumber-1],
    monthFullNames[nextMonthNumber-1]+'-'+monthFullNames[nextSixMonthNumber-1],
    monthFullNames[nextMonthNumber-1]+'-'+monthFullNames[nextTwelveMonthNumber-1]
]


for index, historicallyTrendingMonthData in historicallyTrendingMonthsData.iterrows():
    
    historicallyTrendingMonthDate = historicallyTrendingMonthData['Date']
    historicallyTrendingMonthReturn = historicallyTrendingMonthData['ReturnPercentage']
    historicallyTrendingMonthClose = historicallyTrendingMonthData['Close']
    
    nextMonthDate = historicallyTrendingMonthDate+pd.DateOffset(months=1)
    nextThreeMonthDate = historicallyTrendingMonthDate+pd.DateOffset(months=3)
    nextSixMonthDate = historicallyTrendingMonthDate+pd.DateOffset(months=6)
    nextYearMonthDate = historicallyTrendingMonthDate+pd.DateOffset(months=12)
    
    nextMonthClose = historicTrendDayData[historicTrendDayData['Date'] == nextMonthDate]['Close'].to_list()
    nextThreeMonthClose = historicTrendDayData[historicTrendDayData['Date'] == nextThreeMonthDate]['Close'].to_list()
    nextSixMonthClose = historicTrendDayData[historicTrendDayData['Date'] == nextSixMonthDate]['Close'].to_list()
    nextYearMonthClose = historicTrendDayData[historicTrendDayData['Date'] == nextYearMonthDate]['Close'].to_list()
    
    if ( len(nextMonthClose) == len(nextThreeMonthClose) == len(nextSixMonthClose) == len(nextYearMonthClose) == 1 ):
        nextMonthReturn = round(((nextMonthClose[0]*100)/historicallyTrendingMonthClose) - 100, 2)
        nextThreeMonthReturn = round(((nextThreeMonthClose[0]*100)/historicallyTrendingMonthClose) - 100, 2)
        nextSixMonthReturn = round(((nextSixMonthClose[0]*100)/historicallyTrendingMonthClose) - 100, 2)
        nextYearMonthReturn = round(((nextYearMonthClose[0]*100)/historicallyTrendingMonthClose) - 100, 2) 
        dataList = [
            historicallyTrendingMonthDate.strftime('%Y'), historicallyTrendingMonthReturn,
            nextMonthReturn, nextThreeMonthReturn, nextSixMonthReturn, nextYearMonthReturn
        ]
        historicallyTrendingMonthsDataList = [*historicallyTrendingMonthsDataList, dataList]
    

historicallyTrendingMonthsDataTable = pd.DataFrame(
    historicallyTrendingMonthsDataList, 
    columns=columnNames
).set_index('Year')
historicallyTrendingMonthsDataTable






,January,February,February-April,February-July,February-January
Year,,,,,
2016,-4.79,-7.42,3.78,14.33,13.19
2019,-0.41,-0.14,8.52,2.74,10.62
2020,-1.72,-6.26,-17.67,-7.32,14.16
2021,-2.19,6.31,7.00,15.47,26.92
